In [1]:
import os
import getpass
from dotenv import load_dotenv
load_dotenv()

# Get the API key from the user
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

# Coding Agent with OpenAI

This notebook implements an intelligent coding agent that can:
1. Generate Python code based on natural language tasks
2. Execute the code and capture results
3. Debug code automatically if errors occur
4. Retry up to 3 times to fix issues

The agent uses:
- OpenAI GPT-4 for code generation and debugging
- LangChain for prompt management
- LangGraph for orchestrating the workflow
- Python's exec() for code execution with output capture

## Setup

In [2]:
from typing import TypedDict, List

class AgentState(TypedDict):
    """
    Represents the state of our coding agent.

    Attributes:
        task: The initial user request.
        code: The Python code generated by the LLM.
        result: The output of the executed code.
        error: Any error message captured during execution.
        retries: The number of times the agent has tried to debug the code.
    """
    task: str
    code: str
    result: str
    error: str
    retries: int

## Define Agent State

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- LLM and Prompt Setup ---
# We use temperature=0 for more deterministic and less "creative" code generation.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

code_gen_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are an expert Python programmer. Your role is to write clean, executable Python code to solve a given task.
- Do NOT provide any explanations or markdown formatting like ```python ... ```.
- Only output the raw Python code.
- The code should be self-contained and ready to execute.
- Always include print statements to show the result of the task."""),
    ("human", "Here is the task: {task}")
])

code_generator_chain = code_gen_prompt | llm | StrOutputParser()

# --- Node Definition ---
def generate_code_node(state: AgentState):
    """
    Generates Python code based on the task in the state.
    """
    print("--- Generating Code ---")
    task = state['task']
    code = code_generator_chain.invoke({"task": task})
    return {"code": code, "retries": 0} # Initialize retries

## Node 1: Code Generation

In [4]:
import io
import contextlib

def execute_code_node(state: AgentState):
    """
    Executes the code in the state and captures output or errors.
    """
    print("--- Executing Code ---")
    code = state['code']
    
    try:
        # Use io.StringIO to capture the output of exec
        with io.StringIO() as stdout_capture, contextlib.redirect_stdout(stdout_capture):
            exec(code)
            result = stdout_capture.getvalue()
        
        print(f"Success! Result:\n{result}")
        return {"result": result, "error": None}
    
    except Exception as e:
        error_message = f"Execution failed with error: {e}"
        print(error_message)
        return {"error": error_message, "result": None}

## Node 2: Code Execution

In [5]:
debug_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are an expert Python debugger. Your role is to analyze a piece of code, the original task, and an error message.
Your goal is to fix the code so it can successfully complete the task.
- Do NOT provide any explanations or markdown formatting like ```python ... ```.
- Only output the raw, corrected Python code.
- The code must be self-contained and ready to execute."""),
    ("human",
     """Here is the original task: {task}
     
The following code was generated to solve it:
{code}

But it failed with this error:
{error}

Please provide the corrected code that fixes this error and completes the task.""")
])

debug_chain = debug_prompt | llm | StrOutputParser()

def debug_code_node(state: AgentState):
    """
    Debugs the code based on the error message.
    """
    print("--- Debugging Code ---")
    task = state['task']
    code = state['code']
    error = state['error']
    retries = state.get('retries', 0)
    
    fixed_code = debug_chain.invoke({
        "task": task,
        "code": code,
        "error": error
    })
    
    return {"code": fixed_code, "retries": retries + 1}

## Node 3: Code Debugging

In [6]:
MAX_RETRIES = 3

def should_continue(state: AgentState):
    """
    Determines the next step based on the execution result.
    """
    if state.get("error"):
        if state.get("retries", 0) < MAX_RETRIES:
            print(f"Retrying... (Attempt {state['retries'] + 1})")
            return "debug"  # Go to the debug node
        else:
            print("--- Max retries reached. Ending. ---")
            return "__end__"  # End the graph
    else:
        print("--- Code executed successfully. Ending. ---")
        return "__end__"  # End the graph

## Conditional Edge Logic

In [7]:
from langgraph.graph import StateGraph, END

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("generate", generate_code_node)
workflow.add_node("execute", execute_code_node)
workflow.add_node("debug", debug_code_node)

# Set the entry point
workflow.set_entry_point("generate")

# Add edges
workflow.add_edge("generate", "execute")
workflow.add_conditional_edges(
    "execute",
    should_continue,
    {
        "debug": "debug",
        "__end__": END
    }
)
workflow.add_edge("debug", "execute")

# Compile the graph
app = workflow.compile()

print("Coding agent workflow created successfully!")

Coding agent workflow created successfully!


## Build the Agent Workflow

In [8]:
def run_coding_agent(task: str):
    """
    Main function to run the coding agent.
    """
    print(f"\n{'='*60}")
    print(f"TASK: {task}")
    print(f"{'='*60}\n")
    
    # Initialize the state
    initial_state = {
        "task": task,
        "code": "",
        "result": "",
        "error": "",
        "retries": 0
    }
    
    # Run the agent
    final_state = app.invoke(initial_state)
    
    # Display final results
    print(f"\n{'='*60}")
    print("FINAL RESULTS")
    print(f"{'='*60}")
    print(f"\nGenerated Code:\n{final_state['code']}")
    
    if final_state.get('result'):
        print(f"\nExecution Result:\n{final_state['result']}")
    else:
        print(f"\nExecution Failed with Error:\n{final_state['error']}")
    
    return final_state

## Main Function to Run the Agent

# Example 1: Simple Math Task

In [9]:
# Example 1: Simple calculation
result = run_coding_agent("Calculate the factorial of 7 and print the result")


TASK: Calculate the factorial of 7 and print the result

--- Generating Code ---
--- Executing Code ---
Success! Result:
5040

--- Code executed successfully. Ending. ---

FINAL RESULTS

Generated Code:
n = 7
factorial = 1
for i in range(1, n + 1):
    factorial *= i
print(factorial)

Execution Result:
5040



# Example 2: Data Processing Task

In [10]:
# Example 2: Data processing
result = run_coding_agent("""
Create a list of the first 10 Fibonacci numbers and then:
1. Calculate their sum
2. Find the average
3. Print both the list and the statistics
""")


TASK: 
Create a list of the first 10 Fibonacci numbers and then:
1. Calculate their sum
2. Find the average
3. Print both the list and the statistics


--- Generating Code ---
--- Executing Code ---
Success! Result:
Fibonacci Numbers: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Sum: 88
Average: 8.8

--- Code executed successfully. Ending. ---

FINAL RESULTS

Generated Code:
fib_numbers = [0, 1]
for i in range(2, 10):
    fib_numbers.append(fib_numbers[-1] + fib_numbers[-2])

fib_sum = sum(fib_numbers)
fib_average = fib_sum / len(fib_numbers)

print("Fibonacci Numbers:", fib_numbers)
print("Sum:", fib_sum)
print("Average:", fib_average)

Execution Result:
Fibonacci Numbers: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Sum: 88
Average: 8.8



# Example 3: Error Handling (Intentional Error)

In [11]:
# Example 3: Task that might initially fail (demonstrating debugging)
result = run_coding_agent("""
Read a CSV file called 'data.csv' and print the first 5 rows.
If the file doesn't exist, create sample data with columns: name, age, city
and 5 sample rows, then print it.
""")


TASK: 
Read a CSV file called 'data.csv' and print the first 5 rows.
If the file doesn't exist, create sample data with columns: name, age, city
and 5 sample rows, then print it.


--- Generating Code ---
--- Executing Code ---
Success! Result:
      name  age         city
0    Alice   25     New York
1      Bob   30  Los Angeles
2  Charlie   35      Chicago
3    David   40      Houston
4      Eve   45      Phoenix

--- Code executed successfully. Ending. ---

FINAL RESULTS

Generated Code:
import os
import pandas as pd

filename = 'data.csv'

if not os.path.exists(filename):
    sample_data = {
        'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
        'age': [25, 30, 35, 40, 45],
        'city': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
    }
    df = pd.DataFrame(sample_data)
    df.to_csv(filename, index=False)
else:
    df = pd.read_csv(filename)

print(df.head())

Execution Result:
      name  age         city
0    Alice   25     New York
1      Bob 

# Example 4: Complex Algorithm Implementation

In [ ]:
# Example 4: More complex task
result = run_coding_agent("""
Implement a function to check if a number is prime.
Then use it to find all prime numbers between 1 and 50.
Print the prime numbers and count how many there are.
""")